In [6]:
import os
%pwd
os.chdir('C:\\Users\smith\PycharmProjects\End-to-End-Chest-Cancer-Classification-using-MLflow-DVC')
%pwd

'C:\\Users\\smith\\PycharmProjects\\End-to-End-Chest-Cancer-Classification-using-MLflow-DVC'

In [9]:
import os
from box.exceptions import BoxValueError
import yaml
from cnnClassifier import logger
import json
import joblib
from ensure import ensure_annotations
from box import ConfigBox
from pathlib import Path
from typing import Any
import base64

In [10]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

artifacts_root: artifacts

data_ingestion:
  root_dir: artifacts/data_ingestion
  source_URL: https://drive.google.com/file/d/1z0mreUtRmR-P-magILsDR3T7M6IkGXtY/view?usp=sharing
  local_data_file: artifacts/data_ingestion/data.zip
  unzip_dir: artifacts/data_ingestion

In [21]:
@ensure_annotations
def read_yaml(path_to_yaml: Path) -> ConfigBox: 
    """
    Reads a YAML file and returns its content as a ConfigBox object.
    
    Args:
        path_to_yaml (Path): The path to the YAML file.
        
    Returns:
        ConfigBox: The content of the YAML file as a ConfigBox object.
        
    Raises:
        ValueError: If the YAML file is empty.
        Exception: If any other error occurs while reading the YAML file.
    """
    try:
        with open(path_to_yaml) as yaml_file:
            content = yaml.safe_load(yaml_file)
            logger.info(f"yaml file: {path_to_yaml} loaded successfully")
            print(ConfigBox(content))
            return ConfigBox(content)
            
    except BoxValueError:
        raise ValueError("yaml file is empty")
    except Exception as e:
        raise e #rise e helps to raise the message for this case it print: yaml file is empty

In [26]:
#read_yaml(Path('C:\\Users\smith\PycharmProjects\End-to-End-Chest-Cancer-Classification-using-MLflow-DVC\\config\\data_ingestion_config.yaml'))

import os

file_path = 'C:\\Users\smith\PycharmProjects\End-to-End-Chest-Cancer-Classification-using-MLflow-DVC\config\config.yaml'
#absolute_path = os.path.abspath(file_path)

read_yaml(Path(absolute_path))
x=read_yaml(Path(absolute_path))


[2024-05-25 17:38:55,828: INFO: 3874444000: yaml file: C:\Users\smith\PycharmProjects\End-to-End-Chest-Cancer-Classification-using-MLflow-DVC\config\config.yaml loaded successfully]
{'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://drive.google.com/file/d/1z0mreUtRmR-P-magILsDR3T7M6IkGXtY/view?usp=sharing', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'prepare_base_model': {'root_dir': 'artifacts/prepare_base_model', 'base_model_path': 'artifacts/prepare_base_model/base_model.h5', 'updated_base_model_path': 'artifacts/prepare_base_model/base_model_updated.h5'}, 'training': {'root_dir': 'artifacts/training', 'trained_model_path': 'artifacts/training/model.h5'}}
[2024-05-25 17:38:55,833: INFO: 3874444000: yaml file: C:\Users\smith\PycharmProjects\End-to-End-Chest-Cancer-Classification-using-MLflow-DVC\config\config.yaml loaded successfully]
{'artifacts_root': 'artifacts', 

In [40]:
x.root_dir

BoxKeyError: "'ConfigBox' object has no attribute 'root_dir'"

In [ ]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [23]:
@ensure_annotations
def create_directories(path_to_directories: list, verbose=True):
    """create list of directories

    Args:
        path_to_directories (list): list of path of directories
        ignore_log (bool, optional): ignore if multiple dirs is to be created. Defaults to False.
    """
    for path in path_to_directories:
        os.makedirs(path, exist_ok=True)
        if verbose:
            logger.info(f"created directory at: {path}")


In [ ]:
file_path = 'C:\\Users\smith\PycharmProjects\End-to-End-Chest-Cancer-Classification-using-MLflow-DVC\config\config.yaml'

create_directories([file_path])

In [30]:
def smith():
    print('smith')

In [ ]:
class ConfigurationManager:
    """
    A class that manages the configuration parameters for data ingestion.

    Attributes:
        config_filepath (str): The file path to the configuration file.
        params_filepath (str): The file path to the parameters file.
        config (dict): The configuration parameters.
        params (dict): The parameter values.
    """

    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH
    ):
        """
        Initializes a new instance of the ConfigurationManager class.

        Args:
            config_filepath (str, optional): The file path to the configuration file.
                Defaults to CONFIG_FILE_PATH.
            params_filepath (str, optional): The file path to the parameters file.
                Defaults to PARAMS_FILE_PATH.
        """
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        """
        .In this example, the DataIngestionConfig object tells the system:
        .The data files are located in the "C:/Users/MyName/Data" folder on the computer
        .The data files can be downloaded from the URL "https://example.com/data.zip"
        .The downloaded file should be saved as "data.zip"
        .Once downloaded, the "data.zip" file should be unzipped and placed in the 
            "C:/Users/MyName/Data/Unzipped" folder
        """
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [ ]:
import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [ ]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
     
    def download_file(self)-> str: #for str is the return type for the logger.info
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [ ]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e